## 構文解析（パーサ）の移植
teven John Metsker著「<a href="https://www.amazon.com/Building-Parsers-Java%C2%BF-Steven-Metsker/dp/0201719622">JAVAによるサーバ構築技法（Building Parsers With Java）</a>」に付属しているパーサをpythonに移植しました。

### 字句解析
最初に定義するのは、プログラムの文字列を切り出し、意味のある塊（トークン）に分割する字句解析です。

### トークンタイプ
字句解析の始めに定義するのは、トークンタイプ（TokenType）です。
使用するトークンタイプは、以下の通りです。
- TT_EOF: 文の終わり
- TT_NUMBER: 数字
- TT_WORD: 単語
- TT_SYMBOL: シンボル記号
- TT_QUOTED: クォーテンション

### 字句解析の状態（TokennizerState）
字句解析は解析中の状態（TokennizerState）に沿って処理されます。

字句解析では、次に読み込まれる文字で状態遷移を決定するため、文字列の先読みが必須となります。

「JAVAによるサーバ構築技法」では、JavaのStringReaderを使っており、Pythonへの移植では、peekを使った一文字先読みを行う、PushbackReaderを以下のように定義しました。

```python
import io
import copy

class PushbackReader:
    def __init__(self, s):
        self.peek = None
        self.f = io.StringIO(s)
        if len(s) > 0:
            self.peek = self.f.read(1)
        
    def read(self):
        if self.peek != None:
            c = self.peek
            self.peek = None
            return c
        else:
            c = self.f.read(1)
            if len(c) == 1:
                return c
            else:
                return None
        
    def unread(self, c):
        self.peek = c
```

### Tokenizerの動作確認（ShowTokenizer）
sjm.example.tokens.ShowTokenizerをPythonに移植して、Tokenizerの動作をみてみましょう。

このTokenizerは、Javaのコメントを除いて処理するようになっています。
- // watch out! : 文末まで読み飛ばし
- /* wince */ : コメントの内部も、読み飛ばし

In [1]:
from parser import *

s = """
"It's 123 blast-off!", she said, // watch out!
and <= 3 'ticks' later /* wince */ , it's blast-off!
"""

print(s)

t = Tokenizer(s)
while True:
    tok = t.nextToken()
    if tok == Token.EOF:
        break
    print(f'({tok})')


"It's 123 blast-off!", she said, // watch out!
and <= 3 'ticks' later /* wince */ , it's blast-off!

("It's 123 blast-off!")
(,)
(she)
(said)
(,)
(and)
(<=)
(3)
('ticks')
(later)
(,)
(it's)
(blast-off)
(!)


### トークンナイザ（Tokenizer）
トークンナイザは、内部にASCIIテーブルに対応する状態を保持するchracterStateテーブルを保持しています。

| from | to | state |
|--|--|--|
| 0 | ' ' | whitespaceState |
| 'a' | 'z' | wordState |
| 'A' | 'Z' | wordState |
| 0xc0 | 0xff | wordState |
| '0' | '9' | numberState |
| '-' | '-' | numberState |
| '.' | '.' | numberState |
| '"' | '"' | quoteState |
| "'" | "'" | quoteState |
| '/' | '/' | slashState |

### SymbolState
SymbolState以外のトークン状態は、characterStateテーブルで処理されるのに対し、!=, :-, <=, >=　等のシンボル状態を表すsymbolsに登録して判別するようになっています。

In [4]:
t = Tokenizer("42.001 =~= 42")

t.symbolState.add("=~=")
while True:
    tok = t.nextToken()
    if tok == Token.EOF:
        break
    print(tok)

42.001
=~=
42


### Tokenクラス
Tokenクラスの特徴として、以下点が挙げられます。
- nvalメソッド: トークンを数値として返す
- svalメソッド: トークンを文字列して返す
- ttype: トークンタイプを返す

## 構文解析（パーサ）
「JAVAによるサーバ構築技法」のパーサは、以下の基本的なパーサから構成されています。
- CollectionParser: Sequence, Alternationの共通処理クラス
- Terminal(終端): 
- Word(単語):
- Alternation(選択):
- Empty(空):
- Repetition(繰り返し):
- Sequence(連結):

### ベストマッチの動作
パーサは、構文で与えられた文法で一番長い塊を「ベストマッチ」として返します。

sjm.example.tokens.ShowBestMatchでこの動きを確かめてみましょう。

結果の読み方：
- \['hot', 'steaming', 'hot', 'hot'\]: 切り出されたトークンのスタック（この部分はJava版のShowBestMatchとは、逆の順番で出力されます）
- hot/hot/steaming/hot^coffee: coffeeより前のトークンが処理され、現在位置^でcoffeeを処理していることを示す

## 構文解析（パーサ）の作り方

### アセンブリのスタックの使い方
アセンブリオブジェクトは、スタックとターゲットの２つの作業領域を含みます。
Terminalのサブクラスは、認識済みオブジェクトをスタックに格納します。（格納しない時にはdiscardメソッドを使用）

In [6]:
p = Repetition(Num())
a = p.completeMatch(TokenAssembly("2 4 6 8"))
print(a)

['8', '6', '4', '2']2/4/6/8^


### MiniMathパーサ

演算子"-"だけを認識する算術パーサsjm.examples.MinimathComputeをPythonに移植します。

扱う構文は、以下の通りです。

```
expression = Num minusNum*;
minusNum = '-' Num;
```

### Minimathの式の計算
構文解析とアセンブリが一緒になっているのが、このパーサの特徴の一つです。

各パーサが解析されたら、そのトークンを使って対応するアセンブラが呼び出されます。

以下の例では、数値のアセンブラNumAssemblerとマイナスのアセンブラMinusAssemblerを定義しています。

In [7]:
class NumAssembler(Assembler):
    def workOn(self, a):
        t = a.pop()
        a.push(t.nval)

class MinusAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 - d1
        a.push(d3)

算術パーサを数値の後にマイナス記号と数値が0回以上繰り返します。

最初に作成するのは、連結（Sequence）を生成し、数値（Num）と数値に対するアセンブラ（NumAssembler）をセットしたものを追加します。

同様にマイナス記号と数値の連結を作成し、それを繰り返す（Repetition）を構文に追加します。

In [8]:
e = Sequence()
n = Num()
n.setAssembler(NumAssembler())
e.add(n)

m = Sequence()
m.add(Symbol('-').discard())
m.add(n)
m.setAssembler(MinusAssembler())

e.add(Repetition(m))
out = e.completeMatch(TokenAssembly("25 - 16 - 9"))
print(out.pop())

0


### 優先順位を考慮した構文
四則演算では、積(*)と商(/)は、和(+)と差(-)よりも優先順位が高いですが、これを構文解析で実現する方法が、sjm.examples.arithmetic.ShowArithmeticParserで紹介されています。これをpythonに移植してみます。

アセンブラを考慮して、四則の算術計算を解析するパーサは以下のように定義されます。

```
    expression   = term (plusTerm | minusTerm)*;
    term         = factor (timesFactor | divideFactgor)*;
    plusTerm     = '+' term;
    minusTerm    = '-' term;
    factor       = phrase expFactor | phrase;
    timesFactor  = '*' factor;
    divideFactor = '/' factor;
    expFactor    = '^' factor;
    phrase       = '(' expression ')' | Num;
```

### アセンブラコード
アセンブラコードは、単項処理のNumAssembler以外は2項演算となります。

In [9]:
class NumAssembler(Assembler):
    def workOn(self, a):
        t = a.pop()
        a.push(t.nval)

class PlusAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 + d1
        a.push(d3)

class MinusAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 - d1
        a.push(d3)

class TimesAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 * d1
        a.push(d3)

class DivideAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 / d1
        a.push(d3)

class ExpAssembler(Assembler):
    def workOn(self, a):
        d1 = a.pop()
        d2 = a.pop()
        d3 = d2 ** d1
        a.push(d3)

### ArithmeticParser

In [10]:
class ArithmeticParser:
    @classmethod
    def start(cls):
        return ArithmeticParser().expression()
    def __init__(self):
        self._expression = None
        self._factor = None

    def divideFactor(self):
        s = Sequence()
        s.add(Symbol('/').discard())
        s.add(self.factor())
        s.setAssembler(DivideAssembler())
        return s

    def timesFactor(self):
        s = Sequence()
        s.add(Symbol('*').discard())
        s.add(self.factor())
        s.setAssembler(TimesAssembler())
        return s
        
    def expFactor(self):
        s = Sequence()
        s.add(Symbol('^').discard())
        s.add(self.factor())
        s.setAssembler(ExpAssembler())
        return s
    
    def factor(self):
        if self._factor == None:
            self._factor = Alternation("factor")
            s = Sequence()
            s.add(self.phrase())
            s.add(self.expFactor())

            self._factor.add(s)
            self._factor.add(self.phrase())
        return self._factor

    def plusTerm(self):
        s = Sequence()
        s.add(Symbol('+').discard())
        s.add(self.term())
        s.setAssembler(PlusAssembler())
        return s

    def minusTerm(self):
        s = Sequence()
        s.add(Symbol('-').discard())
        s.add(self.term())
        s.setAssembler(MinusAssembler())
        return s
    
    def phrase(self):
        phrase = Alternation("phrase")
        s = Sequence()
        s.add(Symbol('(').discard())
        s.add(self.expression())
        s.add(Symbol(')').discard())
        phrase.add(s)

        phrase.add(Num().setAssembler(NumAssembler()))
        return phrase
        
    def term(self):
        s = Sequence("term")
        s.add(self.factor())
        a = Alternation()
        a.add(self.timesFactor())
        a.add(self.divideFactor())

        s.add(Repetition(a))
        return s
            
    def expression(self):
        if self._expression == None:
            self._expression = Sequence("expression")
            self._expression.add(self.term())
            
            a = Alternation()
            a.add(self.plusTerm())
            a.add(self.minusTerm())
                    
            self._expression.add(Repetition(a))
        return self._expression
    
    @classmethod
    def value(cls, s):
        ta = TokenAssembly(s)
        a = ArithmeticParser.start().completeMatch(ta)
        d = a.pop()
        return d


テスト用の関数evalを使ってパーサを表示します。

In [11]:
def eval(s, d):
    print(f"Given: {s}\tExpected: {d}\tFound: {ArithmeticParser.value(s)}")

In [12]:
eval("9^2 - 81       ", 0)
eval("7 - 3 - 1      ", 3)
eval("2^1^4          ", 2)
eval("100 - 25*3     ", 25)
eval("100 - 5^2*3    ", 25)
eval("(100 - 5^2) * 3", 225)

Given: 9^2 - 81       	Expected: 0	Found: 0
Given: 7 - 3 - 1      	Expected: 3	Found: 3
Given: 2^1^4          	Expected: 2	Found: 2
Given: 100 - 25*3     	Expected: 25	Found: 25
Given: 100 - 5^2*3    	Expected: 25	Found: 25
Given: (100 - 5^2) * 3	Expected: 225	Found: 225


## 論理型言語の解析

### Logikusの文法
Logikus（簡易Prolog）の文法を以下に示します。

```
    axiom        = structure (ruleDef | Empty);
    structure    = functor('(' commaList(term) ')' | Empty);
    functor      =  '.' | LowercaseWord | QuotedString;
    term         = structure | Num | list | variable;
    variable     = UppercaseWord | '_';

    ruleDef      = ":-" commaList(condition);

    condition    = structure | not | evaluation | comaprison | list;

    not          = "not" structure;

    evaluation   = '#' '(' arg ',' arg ')';
    comparison   = operator '(' arg ',' arg ')';
    arg          = expression | functor;
    operator     = '<' | '>' | '=' | "<=" | ">=" | "!=";

    expression   = phrase ('+' phrase | '-' phrase)*;
    phrase       = factor ('*' factor | '/' factor)*;
    factor       = '(' expression ')' | Num | variable;

    list         = '[' (listContents | Empty) ']';
    listContents = commaList(term) listTail;
    listTail     = ('|' (variable | list)) | Empty;
    
    commaList(p) = p (',' p)*;
```

### Logikusのアセンブラ
Logikusのアセンブラはparser.pyの以下のクラスを参照してください。

- AtomAssembler
- AxiomAssembler
- ComparisonAssembler
- EvaluationAssembler
- ListAssembler
- ListWithTailAssembler
- NotAssembler
- StructureWithTermsAssembler
- VariableAssembler
- ArithmeticAssembler
- AnonymousAssembler

## Logikusのパーサ
Logikusのパーサは、parser.pyのLogikusParserクラスを参照してください。

### Logikusファサードの定義
直接パーザを呼び出して実行するのではなく、中間にLogikusファサードを経由して処理を行うことで、処理を単縦化します。

In [13]:
class LogikusFacade:
    @classmethod
    def axiom(cls, arg):
        if isinstance(arg, TokenString):
            ts = arg
        else:
            s = arg
            ts = TokenString(s)
        p = LogikusParser().axiom()
        o = cls.parse(ts, p, "axiom")
        return o
    
    @classmethod
    def parse(cls, ts, p, _type):
        ta = TokenAssembly(ts)
        out = p.bestMatch(ta)
        if out == None:
            print("reportError")
        if out.hasMoreElements():
            if not out.remainder("") == ";":
                print("reportLeftovers")
        return out.pop()
    
    @classmethod
    def program(cls, s):
        p = Program()
        tss = TokenStringSource(Tokenizer(s), ";")
        while True:
            ts = tss.nextTokenString()
            if ts == None:
                break
            p.addAxiom(cls.axiom(ts))
        return p
    
    @classmethod
    def query(cls, s, _as):
        o = cls.parse(s, LogikusParser.query(), "query")
        if isinstance(o, Fact):
            f = o
            q = Query(_as, f)
        else:
            q = Query(_as, o)
        found = "No"
        while q.canFindNextProof():
            print(q.variables())
            found = "Yes"
        print(found)
    

### Logikusファサードの動作確認

In [14]:
p = LogikusFacade.program('mother(jim, marry);')
print(p)

mother(jim, marry);


In [15]:
LogikusFacade.query('mother(jim, X);', p)

X = marry
Yes


### Logikusの例題を実行
移植したパーサでstarred.txtの例題を実行してみます。

In [16]:
# starred.txtの例題
s = """
starred(jamesCagney, "Muling on the Bountly", 1935);
starred(jamesCagney, "Yankee Doodle Dandy", 1942);
starred(jamesCagney, "Mister Roberts", 1955);
starred(jamesCagney, "Ragtime", 1981);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('starred(jamesCagney, Title, Year);', p)

Title = "Muling on the Bountly", Year = 1935
Title = "Yankee Doodle Dandy", Year = 1942
Title = "Mister Roberts", Year = 1955
Title = Ragtime, Year = 1981
Yes


city.txtの例題を実行し、標高が5000以上の都市を表示してみます。

In [17]:
# city.txtの例題
s ="""
city(abilene, 1718);
city("addis ababa", 8000);
city(denver, 5280);
city(flagstaff, 6970);
city(jacksonville, 8);
city(leadville, 10200);
city(madrid, 1305);
city(richmond, 19);
city(spokane, 1909);
city(wichita, 1305);
highCity(Name) :- city(Name, Alt), >(Alt, 5000);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('highCity(Where);', p)

city(abilene, 1718);
city("addis ababa", 8000);
city(denver, 5280);
city(flagstaff, 6970);
city(jacksonville, 8);
city(leadville, 10200);
city(madrid, 1305);
city(richmond, 19);
city(spokane, 1909);
city(wichita, 1305);
highCity(Name) :- city(Name, Alt), >(Alt, 5000);
Where = "addis ababa"
Where = denver
Where = flagstaff
Where = leadville
Yes


変数による結合の例を試してみます。

In [18]:
s = """
coffe("Launch Mi", french, kenya, 6.95);
coffe("Simple Best", regular, colombia, 5.95);
coffe("Revit", italina, gatemala, 7.95);
coffe("Brimful", regular, kenya, 6.95);
coffe("Smackin", french, colombia, 7.95);

customer("Jim Johnson", 2024);
customer("Jane Jerrod", 2077);
customer("Jasmine Jones", 2093);

order(2024, "Simple Best", 1);
order(2077, "Launch Mi", 3);
order(2077, "Smackin", 3);
order(2024, "Brimful", 2);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('customer(Name, Cnum), order(Cnum, Type, Pounds);', p)

coffe("Launch Mi", french, kenya, 6.95);
coffe("Simple Best", regular, colombia, 5.95);
coffe(Revit, italina, gatemala, 7.95);
coffe(Brimful, regular, kenya, 6.95);
coffe(Smackin, french, colombia, 7.95);
customer("Jim Johnson", 2024);
customer("Jane Jerrod", 2077);
customer("Jasmine Jones", 2093);
order(2024, "Simple Best", 1);
order(2077, "Launch Mi", 3);
order(2077, Smackin, 3);
order(2024, Brimful, 2);
Name = "Jim Johnson", Cnum = 2024, Type = "Simple Best", Pounds = 1
Name = "Jim Johnson", Cnum = 2024, Type = Brimful, Pounds = 2
Name = "Jane Jerrod", Cnum = 2077, Type = "Launch Mi", Pounds = 3
Name = "Jane Jerrod", Cnum = 2077, Type = Smackin, Pounds = 3
Yes


# prologの例題
Logikusパーサを使って「Prologの技芸（art of prolog）」の例題を試してみます。

ちなみにPrologの技芸は私の前職の「株式会社 構造計画研究所」が発行した書籍です。

In [1]:
from parser import *

### 自然数の計算
自然数の「和」をprologで定義します。

自然数の表現にはs(0)が0に１足したことを表すs表現を使用します。

In [4]:
s = """
natural_number(0);
natural_number(s(X)) :- natural_number(X);
plus(0, X, X) :- natural_number(X);
plus(s(X), Y, s(Z)) :- plus(X, Y, Z);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('plus(s(s(s(0))), s(s(0)), Y);', p)

Y = s(s(s(s(s(0)))))
Yes


この結果は、s(s(s(0)))=3, s(s(0))=2なので、s(s(s(s(s(0)))))=5と読みます。

同様に「積」、「べき乗」を追加します。

In [5]:
s = """
natural_number(0);
natural_number(s(X)) :- natural_number(X);
plus(0, X, X) :- natural_number(X);
plus(s(X), Y, s(Z)) :- plus(X, Y, Z);
times(0, X, 0) :- natural_number(X);
times(s(X), Y, Z) :- times(X, Y, XY), plus(XY, Y, Z);
exp(s(N), 0, 0);
exp(0, s(X), s(0));
exp(s(N), X, Y) :- exp(N, X, Z), times(Z, X, Y);
"""
Program.debug = False
p = LogikusFacade.program(s)
LogikusFacade.query('exp(s(s(s(0))), s(s(0)), Yq);', p)

Yq = s(s(s(s(s(s(s(s(0))))))))
Yes


In [6]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('imul(2, [1,-1,0,2,-3], Vs);', p)

Vs = [2, -2, 0, 4, -6]
Yes


In [7]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);

add([], Ys, Ys);
add([X|Xs], [], [X|Xs]);
add([X|Xs], [Y|Ys], [Z|Zs]) :-
	#(Z, X + Y),
	add(Xs, Ys, Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('add([1,2,3], [1,-2,3,-4], Vs)', p)

Vs = [2, 0, 6, -4]
Yes


In [8]:
s = """
imul(_, [], []);
imul(C, [X|Xs], [Z|Zs]) :- #(Z, C * X), imul(C, Xs, Zs);

add([], Ys, Ys);
add([X|Xs], [], [X|Xs]);
add([X|Xs], [Y|Ys], [Z|Zs]) :-
	#(Z, X + Y),
	add(Xs, Ys, Zs);

mul([], _, []);
mul([X|Xs], Ys, Zs) :-
	imul(X, Ys, Zs1),
	mul(Xs, Ys, Zs2),
	add(Zs1, [0|Zs2], Zs);
"""
p = LogikusFacade.program(s)
LogikusFacade.query('mul([1,2,3], [1,-2,3], Vs);', p)

Vs = [1, 0, 2, 0, 9]
Yes


In [9]:
s = """
append([], Z, Z) ;
append([W|X1], Y, [W|Z1]) :- append(X1, Y, Z1);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('append([1], [0], V);', p)


append([], Z, Z);
append([W|X1], Y, [W|Z1]) :- append(X1, Y, Z1);
V = [1, 0]
Yes


### ハノイの塔の例題

In [10]:
s = """
append([], Ys, Ys);
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);

// hanoi(s(0), A, B, C, [[A, to, B]]);
hanoi(0, A, B, C, []);
hanoi(s(N), A, B, C, Moves) :-
	hanoi(N, A, C, B, Ms1),
	hanoi(N, C, B, A, Ms2),
	append(Ms1, [[A, to, B]|Ms2], Moves);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('hanoi(s(s(s(0))), "A", "B", "C", Vs);', p)


append([], Ys, Ys);
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
hanoi(0, A, B, C, []);
hanoi(s(N), A, B, C, Moves) :- hanoi(N, A, C, B, Ms1), hanoi(N, C, B, A, Ms2), append(Ms1, [[A, to, B]|Ms2], Moves);
Vs = [[A, to, B], [A, to, C], [B, to, C], [A, to, B], [C, to, A], [C, to, B], [A, to, B]]
Yes


In [11]:
s = """
city(denver, 5280);
city("jacksonville", 8);
city("test hello", 0);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('city(Name, Heigh);', p)

city(denver, 5280);
city(jacksonville, 8);
city("test hello", 0);
Name = denver, Heigh = 5280
Name = jacksonville, Heigh = 8
Name = "test hello", Heigh = 0
Yes


## 日本語も使える

In [13]:
s = """
"市"("高岡", 100);
"市"("富山", 20);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('"市"(Name, Heigh);', p)

"市"(高岡, 100);
"市"(富山, 20);
Name = 高岡, Heigh = 100
Name = 富山, Heigh = 20
Yes


## デバッグトレース

In [15]:

Program.debug = True
s = """
append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
"""
p = LogikusFacade.program(s)
print(p)
LogikusFacade.query('append(Xs, Ys, [a, b, c]);', p)

Program.debug = False

append([X|Xs], Ys, [X|Zs]) :- append(Xs, Ys, Zs);
append([], Ys, Ys);
append(Xs, Ys, [a, b, c])?
	append([X|Xs], Ys, [X|Zs])	True	Xs = [a|Xs], Ys = Ys, X = a, Zs = [b, c] => append(Xs, Ys, [b, c])
	append([X|Xs], Ys, [X|Zs])	True	Xs = [b|Xs], Ys = Ys, X = b, Zs = [c] => append(Xs, Ys, [c])
	append([X|Xs], Ys, [X|Zs])	True	Xs = [c|Xs], Ys = Ys, X = c, Zs = [] => append(Xs, Ys, [])
	append([X|Xs], Ys, [X|Zs])	False
	append([], Ys, Ys)	True	Xs = [], Ys = [], Ys = []
	Return: Xs = [], Ys = [], Zs = []
	Return: Xs = [c], Ys = [], Zs = [c]
	Return: Xs = [b, c], Ys = [], Zs = [b, c]
	Return: Xs = [a, b, c], Ys = []
Xs = [a, b, c], Ys = []
	append([], Ys, Ys)	True	Xs = [], Ys = [c], Ys = [c]
	Return: Xs = [], Ys = [c], Zs = [c]
Xs = [a, b], Ys = [c]
	append([], Ys, Ys)	True	Xs = [], Ys = [b, c], Ys = [b, c]
	Return: Xs = [], Ys = [b, c], Zs = [b, c]
Xs = [a], Ys = [b, c]
	append([], Ys, Ys)	True	Xs = [], Ys = [a, b, c], Ys = [a, b, c]
	Return: Xs = [], Ys = [a, b, c]
Xs = [], Ys = [a, b, c]
Ye